# Real Estate Price Prediction: End-to-End Regression

This notebook builds a complete regression pipeline to predict house prices from the provided dataset (`house_data.csv`). It includes data exploration, feature engineering, model training and comparison, error analysis, feature importance, and interactive visuals.

In [1]:
# Imports & settings
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
import joblib

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_dark'
sns.set_theme(style='whitegrid')

from sklearn.model_selection import train_test_split, RepeatedKFold, cross_validate, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import (r2_score, mean_squared_error, mean_absolute_error,
                            mean_absolute_percentage_error)
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.inspection import permutation_importance

# Optional: SHAP for explainability (may be skipped if not installed)
try:
    import shap
    shap_available = True
except Exception:
    shap_available = False

print('Libraries imported. SHAP available:', shap_available)

Libraries imported. SHAP available: True


## Load dataset
We'll load `house_data.csv` from the workspace root.

In [7]:
# Load dataset
data_path = 'house_data.csv'
assert os.path.exists(data_path), f'File not found: {data_path}'
df = pd.read_csv(data_path)
print(df.shape)
df.head()

(21613, 21)


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## Quick data overview
Structure, typing, basic stats, and missing values.

In [3]:
display(df.info())
display(df.describe(include='all').T)
missing = df.isna().sum().sort_values(ascending=False)
missing[missing>0]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  int64  
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

None

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,21613.0,NaN,NaN,NaN,4580301520.864988,2876565571.312057,1000102.0,2123049194.0,3904930410.0,7308900445.0,9900000190.0
date,21613,372,20140623T000000,142,NaN,NaN,NaN,NaN,NaN,NaN,NaN
price,21613.0,NaN,NaN,NaN,540088.141767,367127.196483,75000.0,321950.0,450000.0,645000.0,7700000.0
bedrooms,21613.0,NaN,NaN,NaN,3.370842,0.930062,0.0,3.0,3.0,4.0,33.0
bathrooms,21613.0,NaN,NaN,NaN,2.114757,0.770163,0.0,1.75,2.25,2.5,8.0
sqft_living,21613.0,NaN,NaN,NaN,2079.899736,918.440897,290.0,1427.0,1910.0,2550.0,13540.0
sqft_lot,21613.0,NaN,NaN,NaN,15106.967566,41420.511515,520.0,5040.0,7618.0,10688.0,1651359.0
floors,21613.0,NaN,NaN,NaN,1.494309,0.539989,1.0,1.0,1.5,2.0,3.5
waterfront,21613.0,NaN,NaN,NaN,0.007542,0.086517,0.0,0.0,0.0,0.0,1.0
view,21613.0,NaN,NaN,NaN,0.234303,0.766318,0.0,0.0,0.0,0.0,4.0


Series([], dtype: int64)

## Parse and engineer features
We'll parse the `date` column, extract `sale_year` and `sale_month`, and prepare feature sets.

In [10]:
# Parse date like 20141013T000000
if 'date' in df.columns:
    try:
        df['date'] = pd.to_datetime(df['date'], format='%Y%m%dT%H%M%S', errors='coerce')
    except Exception:
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['sale_year'] = df['date'].dt.year
    df['sale_month'] = df['date'].dt.month
else:
    df['sale_year'] = np.nan
    df['sale_month'] = np.nan

# Categorical handling for zipcode
if 'zipcode' in df.columns:
    df['zipcode'] = df['zipcode'].astype(str)

# Define target and candidate features
target = 'price'
drop_cols = [c for c in ['id', 'date'] if c in df.columns]
features = [c for c in df.columns if c not in drop_cols + [target]]

numeric_features = [c for c in features if pd.api.types.is_numeric_dtype(df[c])]
categorical_features = [c for c in features if c not in numeric_features]
print('Numeric:', len(numeric_features), 'Categorical:', len(categorical_features))
numeric_features[:10], categorical_features[:10]

Numeric: 19 Categorical: 1


(['bedrooms',
  'bathrooms',
  'sqft_living',
  'sqft_lot',
  'floors',
  'waterfront',
  'view',
  'condition',
  'grade',
  'sqft_above'],
 ['zipcode'])

## Exploratory visualizations (interactive)
A few interactive Plotly visuals to understand distributions and relationships.

In [5]:
# Price distribution
fig = px.histogram(df, x='price', nbins=60, title='Price Distribution')
fig.update_traces(marker_color='#00CC96')
fig.show()

# sqft_living vs price, colored by grade
if 'sqft_living' in df.columns:
    fig = px.scatter(df, x='sqft_living', y='price', color=df.get('grade', None),
                     title='sqft_living vs price (colored by grade)',
                     opacity=0.6, trendline='ols')
    fig.show()

# Correlation heatmap (numeric only)
corr = df[numeric_features + ([target] if target in df.columns else [])].corr()
fig = px.imshow(corr, color_continuous_scale='Viridis', title='Correlation Heatmap')
fig.update_layout(height=700)
fig.show()

## Train/validation split and preprocessing
We'll standardize numeric features (for linear models) and one-hot encode categoricals.

In [11]:
# Split data
X = df[features].copy()
y = df[target].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessors
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    # sklearn >=1.2 uses 'sparse_output' instead of removed 'sparse'
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

print('Train/Test shapes:', X_train.shape, X_test.shape)

Train/Test shapes: (17290, 20) (4323, 20)


## Build and compare models
We'll evaluate Linear Regression, Ridge, Random Forest, and Gradient Boosting via cross-validation.

In [12]:
# Define model pipelines
models = {
    'LinearRegression': Pipeline(steps=[('preprocess', preprocessor), ('model', LinearRegression())]),
    'Ridge': Pipeline(steps=[('preprocess', preprocessor), ('model', Ridge(alpha=10.0))]),
    'RandomForest': Pipeline(steps=[('preprocess', preprocessor), ('model', RandomForestRegressor(n_estimators=300, random_state=42))]),
    'GradientBoosting': Pipeline(steps=[('preprocess', preprocessor), ('model', GradientBoostingRegressor(random_state=42))]),
}

# Cross-validate with multiple metrics
cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=42)
rows = []
for name, pipe in models.items():
    scores = cross_validate(pipe, X_train, y_train, cv=cv,
                            scoring={'r2': 'r2', 'neg_mse': 'neg_mean_squared_error', 'neg_mae': 'neg_mean_absolute_error'},
                            n_jobs=-1, return_train_score=False)
    r2_mean = scores['test_r2'].mean()
    rmse_mean = np.sqrt((-scores['test_neg_mse']).mean())
    mae_mean = (-scores['test_neg_mae']).mean()
    rows.append({'model': name, 'CV_R2': r2_mean, 'CV_RMSE': rmse_mean, 'CV_MAE': mae_mean})
cv_results = pd.DataFrame(rows).sort_values('CV_RMSE')
cv_results

,model,CV_R2,CV_RMSE,CV_MAE
3,GradientBoosting,0.875404,127431.702005,75307.194973
2,RandomForest,0.874364,128363.975826,68984.369833
0,LinearRegression,0.806931,159414.853502,95291.556855
1,Ridge,0.805193,160120.574013,95547.056737


## Fit best model and test performance
We'll pick the CV-best model (by RMSE), fit on the training set, and evaluate on the test set.

In [16]:
best_name = cv_results.iloc[0]['model']
best_pipeline = models[best_name]
best_pipeline.fit(X_train, y_train)

preds = best_pipeline.predict(X_test)
r2 = r2_score(y_test, preds)
rmse = mean_squared_error(y_test, preds, squared=False)
mae = mean_absolute_error(y_test, preds)
mape = mean_absolute_percentage_error(y_test, preds)
summary = pd.DataFrame([{'Model': best_name, 'Test_R2': r2, 'Test_RMSE': rmse, 'Test_MAE': mae, 'Test_MAPE': mape}])
summary

,Model,Test_R2,Test_RMSE,Test_MAE,Test_MAPE
0,GradientBoosting,0.864208,143277.874209,78798.024715,0.146881


## Error analysis and cool visuals
Residual plots, predicted vs actual, and error breakdowns.

In [14]:
residuals = y_test - preds
viz_df = pd.DataFrame({'Actual': y_test.values, 'Predicted': preds})
viz_df['Residual'] = viz_df['Actual'] - viz_df['Predicted']

# Predicted vs Actual
fig = px.scatter(viz_df, x='Actual', y='Predicted', title=f'Predicted vs Actual ({best_name})',
                 opacity=0.7, trendline='ols')
fig.add_shape(type='line', x0=viz_df['Actual'].min(), y0=viz_df['Actual'].min(),
              x1=viz_df['Actual'].max(), y1=viz_df['Actual'].max(),
              line=dict(color='cyan', dash='dash'))
fig.show()

# Residual distribution
fig = px.histogram(viz_df, x='Residual', nbins=60, title='Residuals Distribution')
fig.update_traces(marker_color='#AB63FA')
fig.show()

# Residuals vs Predicted
fig = px.scatter(viz_df, x='Predicted', y='Residual', title='Residuals vs Predicted', opacity=0.6)
fig.add_hline(y=0, line_dash='dash', line_color='orange')
fig.show()

# Error by zipcode (if available)
if 'zipcode' in X_test.columns:
    tmp = X_test.copy()
    tmp['Residual'] = residuals.values
    by_zip = tmp.groupby('zipcode')['Residual'].apply(lambda s: s.abs().mean()).sort_values(ascending=False).head(15)
    fig = px.bar(by_zip, title='Top 15 ZIP codes by Mean Absolute Residual', labels={'value':'Mean |Residual|','index':'zipcode'})
    fig.show()

## Feature importance
We'll compute model-dependent importance (coefficients or tree importances) and permutation importance.

In [17]:
# Recover feature names after preprocessing
def get_feature_names(preprocessor, numeric_features, categorical_features):
    num_features = list(numeric_features)
    try:
        ohe = preprocessor.named_transformers_['cat'].named_steps['onehot']
        try:
            # Newer sklearn
            ohe_names = list(ohe.get_feature_names_out(categorical_features))
        except Exception:
            # Fallback for very old sklearn without get_feature_names_out
            ohe_names = []
            for i, cat in enumerate(categorical_features):
                ohe_names += [f"{cat}_{c}" for c in ohe.categories_[i]]
    except Exception:
        ohe_names = list(categorical_features)
    return num_features + ohe_names

feature_names = get_feature_names(best_pipeline.named_steps['preprocess'], numeric_features, categorical_features)

# Model-dependent importance
model = best_pipeline.named_steps['model']
importances = None
if hasattr(model, 'feature_importances_'):
    importances = model.feature_importances_
elif hasattr(model, 'coef_'):
    coef = model.coef_
    importances = np.abs(coef)

if importances is not None:
    imp_df = pd.DataFrame({'feature': feature_names[:len(importances)], 'importance': importances})
    imp_df = imp_df.sort_values('importance', ascending=False).head(25)
    fig = px.bar(imp_df, x='importance', y='feature', orientation='h', title=f'Feature Importance ({best_name})')
    fig.update_layout(height=700)
    fig.show()
else:
    print('Model does not provide native feature importance; see permutation importance below.')

# Permutation importance on the test set works at the ORIGINAL feature level (pre-transform)
perm = permutation_importance(best_pipeline, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)
perm_names = list(X_test.columns)  # original feature names in correct order
perm_imp = pd.DataFrame({'feature': perm_names, 'importance': perm.importances_mean})
perm_imp = perm_imp.sort_values('importance', ascending=False).head(25)
fig = px.bar(perm_imp.sort_values('importance'), x='importance', y='feature', orientation='h',
            title='Permutation Importance (Test Set, Original Features)')
fig.update_layout(height=700)
fig.show()

## Optional: SHAP explainability
If SHAP is available, show a beeswarm summary for a sample; otherwise this cell will be skipped.

In [18]:
if shap_available:
    # Use a small sample for speed
    sample_idx = np.random.RandomState(42).choice(len(X_test), size=min(1000, len(X_test)), replace=False)
    X_sample = X_test.iloc[sample_idx]

    # For tree models, TreeExplainer is fast; fallback to KernelExplainer otherwise
    try:
        explainer = shap.Explainer(best_pipeline)
        shap_values = explainer(X_sample)
        shap.plots.beeswarm(shap_values, max_display=20)
    except Exception as e:
        print('SHAP plotting skipped:', e)
else:
    print('SHAP is not available. You can install it with: pip install shap')

SHAP plotting skipped: The passed model is not callable and cannot be analyzed directly with the given masker! Model: Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['bedrooms', 'bathrooms',
                                                   'sqft_living', 'sqft_lot',
                                                   'floors', 'waterfront',
                                                   'view', 'condition', 'grade',
                                                   'sqft_above',
                                                   'sqft_basement', 'yr_bu

## Save the trained model
We persist the best model pipeline (with preprocessing) using joblib.

In [19]:
model_to_save = best_pipeline
save_path = 'best_real_estate_model.joblib'
joblib.dump(model_to_save, save_path)
print('Saved to', save_path)

Saved to best_real_estate_model.joblib


## Inference example
How to load the model and score new data.

In [20]:
loaded = joblib.load('best_real_estate_model.joblib')
# Create a single-row example using medians/modes from training data
row = {}
for c in numeric_features:
    row[c] = float(X_train[c].median())
for c in categorical_features:
    row[c] = str(X_train[c].mode().iloc[0]) if len(X_train[c].mode()) > 0 else ''
example = pd.DataFrame([row])
pred_example = loaded.predict(example)[0]
print('Predicted price for example row:', round(pred_example, 2))

Predicted price for example row: 483070.12


## Conclusions
- We prepared data, engineered features, and compared several regression models.
- We evaluated performance via cross-validation and a held-out test set.
- Feature importance and permutation importance highlighted key drivers.
- Interactive Plotly visuals made exploration and error analysis intuitive.

Next ideas: try more models (XGBoost/LightGBM), richer feature engineering (interactions, logs), and spatial features.